In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import ndjson
import os
from torch.utils.data import Dataset, DataLoader

IMAGESIZE = 28



In [4]:
def parseSimplifiedDrawings(fileName):
        drawings = []
        with open(fileName, 'r') as fileStream:
            for obj in ndjson.load(fileStream):
                drawings.append(obj)
        return drawings


In [5]:
class ImageDataLoader(Dataset):
    """Quick Draw and Image dataset."""

    def __init__(self, length, img_dir="images", data_dir="data"):
        """
        Arguments:
            len: length of data requested
            img_dir (string): path to images
            data_dir (string): Directory with all the images.
        """
        self.absolute_path = "/Users/arthurjakobsson/Documents/10617/Project/"
        img_path = os.path.join(self.absolute_path, img_dir)
        data_path = os.path.join(self.absolute_path, data_dir)
        img_list = sorted(os.listdir(img_path))
        data_list = sorted(os.listdir(data_dir))
        # img_list = img_list[0:length] #temporary cut it short
        # data_list = data_list[0:length] #temporary cut it short

        print(img_list)
        print(data_list)
        self.img_npy = np.vstack([np.load(os.path.join(img_path, fname)) for fname in img_list])
        print(self.img_npy.shape)
        allDrawings = []
        for fname in data_list:
            drawings = parseSimplifiedDrawings("data/"+ fname)
            allDrawings += drawings
            print(fname)
        dicts = np.array([allDrawings]).T

        self.img_npy = np.array(self.img_npy)
        self.data_dir = data_list

        self.finalData = np.hstack([self.img_npy, dicts])
        print(self.finalData.shape)
        self.length = self.finalData.shape[0]
        np.save("AllData.npy", self.finalData)


    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if idx >= self.__len__():
            print("Index too large for {}-sample dataset".format(self.__len__()))
            return

        dataPoint = self.data[idx]
        xData = dataPoint[:(IMAGESIZE*IMAGESIZE)]
        dataInfo = dataPoint[IMAGESIZE*IMAGESIZE]
        countryCode = dataInfo["countrycode"]
        return xData, countryCode



        # img_name = os.path.join(self.root_dir,
        #                         self.landmarks_frame.iloc[idx, 0])
        # image = io.imread(img_name)
        # landmarks = self.landmarks_frame.iloc[idx, 1:]
        # landmarks = np.array([landmarks], dtype=float).reshape(-1, 2)
        # sample = {'image': image, 'landmarks': landmarks}

        # if self.transform:
        #     sample = self.transform(sample)

        return sample

dataloader = ImageDataLoader(0)


['The Eiffel Tower.npy', 'The Great Wall of China.npy', 'The Mona Lisa.npy', 'aircraft carrier.npy', 'airplane.npy', 'alarm clock.npy', 'ambulance.npy', 'angel.npy', 'animal migration.npy', 'ant.npy', 'anvil.npy', 'apple.npy', 'arm.npy', 'asparagus.npy', 'axe.npy', 'backpack.npy', 'banana.npy', 'bandage.npy', 'barn.npy', 'baseball bat.npy', 'baseball.npy', 'basket.npy', 'basketball.npy', 'bat.npy', 'bathtub.npy', 'beach.npy', 'bear.npy', 'beard.npy', 'bed.npy', 'bee.npy', 'belt.npy', 'bench.npy', 'bicycle.npy', 'binoculars.npy', 'bird.npy', 'birthday cake.npy', 'blackberry.npy', 'blueberry.npy', 'book.npy', 'boomerang.npy', 'bottlecap.npy', 'bowtie.npy', 'bracelet.npy', 'brain.npy', 'bread.npy', 'bridge.npy', 'broccoli.npy', 'broom.npy', 'bucket.npy', 'bulldozer.npy', 'bus.npy', 'bush.npy', 'butterfly.npy', 'cactus.npy', 'cake.npy', 'calculator.npy', 'calendar.npy', 'camel.npy', 'camera.npy', 'camouflage.npy', 'campfire.npy', 'candle.npy', 'cannon.npy', 'canoe.npy', 'car.npy', 'carrot.

: 

In [ ]:

class OurCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.LogSoftmax(dim=1)
        self.drop1 = nn.Dropout(0.3)

        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.LogSoftmax(dim=1)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        self.flat = nn.Flatten()

        self.fc3 = nn.Linear(8192, 512)
        self.act3 = nn.LogSoftmax(dim=1)
        self.drop3 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(512, 10)

    def forward(self, x):
        # input 3x32x32, output 32x32x32
        x = self.act1(self.conv1(x))
        x = self.drop1(x)
        # input 32x32x32, output 32x32x32
        x = self.act2(self.conv2(x))
        # input 32x32x32, output 32x16x16
        x = self.pool2(x)
        # input 32x16x16, output 8192
        x = self.flat(x)
        # input 8192, output 512
        x = self.act3(self.fc3(x))
        x = self.drop3(x)
        # input 512, output 10
        x = self.fc4(x)
        return x


transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

model = OurCNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def prepare_inputs(self, sampled_batch, isNumpy=False):
    def prep(x):
        return x.to(ptu.device).float()
    # obs, hvals, newHVal = sampled_batch
    # if isNumpy:
    #     obs, hvals, newHVal = ptu.from_numpy(obs), ptu.from_numpy(hvals), ptu.from_numpy(newHVal)
    if isNumpy:
        obs, backward_d, helper_bd, nearby_agents, state = \
                    [ptu.from_numpy(x) for x in sampled_batch]
    else:
        obs, backward_d, helper_bd, nearby_agents, state = sampled_batch

    #TODO Fix everything below, things should be loaded in properly (deal with nearby agents)

    inputs = torch.stack([obs, backward_d, helper_bd[:,0], helper_bd[:,1], helper_bd[:,2], helper_bd[:,3]], dim=1) # (B,6,D,D) #TODO undo hardcoded 4 helper agents
    if isinstance(self.my_model, simple_MLP) or \
            isinstance(self.my_model, simple_MLP2): # Flatten if model is an MLP
        inputs = inputs.reshape((obs.shape[0], -1)) # (B,6,D,D) -> (B,6*D*D)

n_epochs = 1
for epoch in range(n_epochs):
    for inputs, labels in trainloader:
        # forward, backward, and then weight update
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    acc = 0
    count = 0
    for inputs, labels in testloader:
        y_pred = model(inputs)
        acc += (torch.argmax(y_pred, 1) == labels).float().sum()
        count += len(labels)
    acc /= count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

torch.save(model.state_dict(), "cifar10model.pth")
